# BNL Development Notebook

This notebook is for developing and testing the BNL package.


In [27]:
from bnl import data
from pathlib import Path

TRACK_ID = "8"
FIXTURES_DIR = Path("../tests/fixtures")
TEST_MANIFEST = FIXTURES_DIR / "test_manifest.csv"

dataset = data.Dataset(TEST_MANIFEST)

dataset

In [34]:
slm_ds = bnl.data.Dataset(manifest_path="~/data/salami/metadata.csv")

In [36]:
slm_ds.manifest

,track_id,has_annotation_adobe-mu1gamma1,has_annotation_adobe-mu1gamma9,has_annotation_adobe-mu5gamma5,has_annotation_reference,has_audio_mp3
track_id,,,,,,
2,2,True,True,True,True,True
3,3,True,True,True,True,True
4,4,True,True,True,True,True
5,5,True,True,True,True,True
6,6,True,True,True,True,True
...,...,...,...,...,...,...
1650,1650,True,True,True,True,True
1652,1652,True,True,True,True,True
1653,1653,True,True,True,True,True


In [33]:
dataset[2]

Track(track_id='2', num_assets=5, source='cloud')

In [23]:
%load_ext autoreload
%autoreload 2

import bnl
from random import choice

slm_ds = bnl.data.Dataset(manifest_path="~/data/salami/metadata.csv")
# slm_ds = bnl.data.Dataset()
track = slm_ds[11]

available_refs = list(track.refs)
available_ests = list(track.ests)
print(available_refs, available_ests)

est = track.ests[choice(available_ests)]
ref = track.refs[choice(available_refs)]

ref.plot().show()
est.plot().show()



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ValueError: All layers must have the same end time.

In [27]:
from bnl import Segment as S, MultiSegment as MS

mock_empty_layer = S([ref.start, ref.end], [None], 'empty')
mock_layer = S(ref[1].boundaries, [lab*2 for lab in ref[1].labels], 'redundant')
mock_ref = MS([mock_empty_layer] + ref.layers + [mock_layer], name='mock_ref')

mock_ref.plot().show()
mock_ref.prune_layers().plot().show()


Skipping layer redundant


In [19]:
est1 = est[0].copy()
est1.name = "5"
est1.boundaries[-1] = bnl.Boundary(225)

In [22]:
est1

TS(B(0.0)-B(222.04082), 5)

In [4]:
est.prune_layers(relabel=False).plot()

In [3]:
# First figure: we need to show for T-measure, what's it's deal with monotonicity.
import mir_eval
import numpy as np

aligned_est = ref.align_layers(est)
mono_est_count = aligned_est.to_contour('count').to_hierarchy().to_multisegment()
mono_est_depth = aligned_est.to_contour('depth').to_hierarchy().to_multisegment()
mono_est_prob = aligned_est.to_contour('prob').to_hierarchy().to_multisegment()


In [29]:
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.04,
    subplot_titles=["Reference", "Estimated Raw", "Count", "Depth"],
    row_heights=[0.1, 0.3, 0.2, 0.2],
)
[fig.add_trace(labeld_timespans, row=1, col=1) for labeld_timespans in ref.plot().data]
[fig.add_trace(labeld_timespans, row=2, col=1) for labeld_timespans in aligned_est.plot().data]
[fig.add_trace(labeld_timespans, row=3, col=1) for labeld_timespans in mono_est_count.plot().data]
[fig.add_trace(labeld_timespans, row=4, col=1) for labeld_timespans in mono_est_depth.plot().data]
fig.update_layout(barmode="overlay", legend_visible=False, height=1000, width=800)
fig.update_xaxes(autorangeoptions=dict(minallowed=ref.start.time, maxallowed=ref.end.time))
fig.show()

In [30]:
mono_est_depth.plot()

In [31]:
aligned_est.plot()

In [53]:
import mir_eval
import numpy as np
from warnings import filterwarnings
filterwarnings("ignore")

mono_est_depth = aligned_est.to_contour('depth').to_multisegment()

a = mir_eval.hierarchy.tmeasure(ref.itvls, aligned_est.itvls[2:6], transitive=True)
b = mir_eval.hierarchy.tmeasure(ref.itvls, mono_est_depth.itvls[2:6], transitive=True)
c = mir_eval.hierarchy.tmeasure(ref.itvls, aligned_est.itvls[2:6], transitive=False)
d = mir_eval.hierarchy.tmeasure(ref.itvls, mono_est_depth.itvls[2:6], transitive=False)


print(np.array([a, b, c, d]))

[[0.66127564 0.51144347 0.57678792]
 [0.69308449 0.53833029 0.60598327]
 [0.4956261  0.39625926 0.44040734]
 [0.21179284 0.40614068 0.27840434]]


In [77]:
from mir_eval.hierarchy import tmeasure

annos = [
    ref.scrub_labels(), 
    MS(aligned_est[2:6], 'Raw').scrub_labels(), 
    MS(mono_est_depth[2:6], 'Mono')
]

fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True,
    subplot_titles=["Reference", "Raw", "Mono"]
)
fig.update_layout(
    barmode="overlay", legend_visible=False, height=650, width=650, 
    margin=dict(l=10, r=10, t=50, b=10)
)
fig.update_xaxes(
    autorangeoptions=dict(minallowed=ref.start.time, maxallowed=ref.end.time)
)

for row, anno in enumerate(annos, start=1):
    if row != 1:
        print(f"{anno}: reduced (tran=False)\n  {tmeasure(ref.itvls, anno.itvls)}")
        print(f"{anno}: full (tran=True)\n  {tmeasure(ref.itvls, anno.itvls, transitive=True)}")
    for bar_trace in anno.plot().data:
        fig.add_trace(bar_trace, row=row, col=1)

fig.show()

Raw: reduced (tran=False)
  (0.49562610225667814, 0.3962592551085229, 0.44040734264942816)
Raw: full (tran=True)
  (0.6612756416093231, 0.5114434653772998, 0.576787917412396)
Mono: reduced (tran=False)
  (0.21179283577992608, 0.40614068329499714, 0.27840434087284016)
Mono: full (tran=True)
  (0.6930844877333746, 0.5383302933543088, 0.60598326629029)


In [69]:
[attr for attr in dir(MS) if not attr.startswith("__")]

['align_layers',
 'duration',
 'from_json',
 'itvls',
 'labels',
 'name',
 'plot',
 'to_contour']

In [ ]:
from bnl import Segment as S
layer_itvls = annos[1].itvls[0]
layer = S.from_itvls(layer_itvls, labels=[""] * len(layer_itvls))
layer.plot()


ValueError: Number of labels must be one less than the number of boundaries.

In [38]:
from bnl import MultiSegment as MS
MS(aligned_est[2:]).plot()

In [ ]:
# Study the effect of monotonic casting and T-measure